In [ ]:
import random
import copy
import networkx as nx
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import math
import pickle
import os
import time
from datetime import datetime
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
random.seed(datetime.now().timestamp())

In [ ]:
!unzip /content/gdrive/MyDrive/Networks/test_cases.zip -d /

In [ ]:
num_of_nodes =  [20,25,30,35,40,45,50,55,60]
test_case_num = 30

#defining network parameters
GATEWAY_ID = 0
spread_factor = range(7,13)

In [ ]:
#initializing chromosomes
population_size = 44
max_generations = 500
init_priority = (-100,100)
large_positive_value = 10**6
crossover_rate = 0.62
tau_max = 0.3
mutation_rate = 0.12
C = 0.5
alpha = 1
beta = 2

In [ ]:
class Population:
  def __init__(self, objVal, fitness, chromosome):
    self.objVal = objVal
    self.fitness = fitness
    self.chromosome = chromosome

  def __str__(self):
    return f'''
              Chromosome = {self.chromosome}\n
              fitness    = {self.fitness}\n
              objective  = {self.objVal}\n
            '''

In [ ]:
def dist(a,b):
  x1,y1 = a
  x2,y2 = b
  return math.sqrt((x2-x1)**2+(y2-y1)**2)

In [ ]:
def bsearch(arr,target):
  low = 0; high = len(arr)-1
  while(low < high):
      mid = math.floor((low + high)/2)
      if(arr[mid] >= target):
        high = mid
      else:
        low = mid+1
  return low

In [ ]:
def read_data(path, isGraph):
  if isGraph:
    G = nx.read_gml(path,destringizer=int)
    return G
  with open(path,'rb') as file:
    data = pickle.load(file)
    file.close()
    return data

In [ ]:
#Integer-Valued-Priority-based-encoding
def decode_path(G, start_node, chromosome):

  #list with a single element start_node
  nodes = [start_node]
  indices = []
  valid_path = True
  while nodes[-1] != GATEWAY_ID:

    #Set of neighours in G of last element in nodes
    neighours = G.adj[nodes[-1]]

    #Set of elements in neighours not in nodes
    allowed = [ele for ele in neighours if ele not in nodes]

    #if allowed is empty means path got stuck.
    if len(allowed) == 0:
      nodes.append(GATEWAY_ID)
      valid_path = False
    else:
      #last element of nodes list
      prevNode = nodes[-1]

      #node with maximum priority in allowed
      subset = {key : chromosome[key] for key in chromosome.keys() if key in allowed}
      nextNode = max(subset ,key = lambda x : subset[x])

      #Add nextNode to nodes
      nodes.append(nextNode)

      #paralled edge indicator for node prevNode
      r = abs(chromosome[prevNode]) - int(abs(chromosome[prevNode]))
      currentIndex = math.floor(r*6)+1
    
    #add currentIndex to indices
    indices.append(currentIndex+6)

  return [valid_path,nodes,indices]

In [ ]:
def evaluate_population(population ,generation_counter, edge_nodes, G, network_data):
  for individual in population:
    obj_val = 0
    for f in edge_nodes:
      valid, nodes, indices = decode_path(G, f, individual.chromosome)
      #adding a penalty if invalid path is returned
      if not valid:
        obj_val = obj_val + large_positive_value
        continue

      indices.append(7)
      a = [i for i in nodes[:-1]]
      b = [j for j in nodes[1:]]
      path = [(i,j,k) for i,j,k in zip(a,b,indices)]
      #calculate cost of path and adding link capacity penalty
      total_delay = 0
      for i,j,k in path:
        obj_val = obj_val + network_data['cst'][f,i,j,k]
        if network_data['max_edge_data_rate'][k]-network_data['data_rate'][f,i,j,k] < 0:
          obj_val = obj_val + network_data['cst'][f,i,j,k] + ((C*generation_counter)**alpha)*((network_data['max_edge_data_rate'][k]-network_data['data_rate'][f,i,j,k])**beta) 
        total_delay = total_delay + network_data['edge_delays'][f,i,j] + network_data['tx_time'][k] + network_data['eh_time'][i,k]
      
      #Adding delay constraint penalty
      if network_data['max_flow_delays'][f]-total_delay < 0:
        obj_val = obj_val + ((C*generation_counter)**alpha)*((network_data['max_flow_delays'][f]-total_delay)**beta)
    individual.objVal = obj_val
    individual.fitness = 1/obj_val

In [ ]:
def roulette_wheel_sel(population):
  new_population = []
  probability = []
  total_fitness = 0
  for p in population:
    total_fitness = total_fitness + p.fitness
  for p in population:
    probability.append(p.fitness/total_fitness)
  
  cumulative_prob = [probability[0]]
  for k in range(1,population_size):
    cumulative_prob.append(cumulative_prob[-1] + probability[k])

  for k in range(population_size):
    rand_num = random.uniform(0,1);
    idx = bsearch(cumulative_prob, rand_num)
    new_population.append(copy.deepcopy(population[idx]))
  return new_population

In [ ]:
def perform_crossover(p1,p2,population):
  N_DEVICES = len(population[0].chromosome)
  num_genes = random.randint(1,N_DEVICES-1)
  pos_selected = random.sample(range(N_DEVICES),num_genes)
  parent1_chromo = population[p1].chromosome
  parent2_chromo = population[p2].chromosome
  offspring1_chromo = {i:-1 for i in range(N_DEVICES)}
  offspring2_chromo = {i:-1 for i in range(N_DEVICES)}

  i = 0
  j = 0
  for idx in range(len(offspring1_chromo)):
    if idx in pos_selected:
      offspring1_chromo[idx] = parent1_chromo[idx]
      offspring2_chromo[idx] = parent2_chromo[idx]
    else:
      while i < len(parent2_chromo) and parent2_chromo[i] in offspring1_chromo:
        i = i+1
      while j < len(parent1_chromo) and parent1_chromo[j] in offspring2_chromo:
        j = j+1

      offspring1_chromo[idx] = parent2_chromo[i]
      offspring2_chromo[idx] = parent1_chromo[j]
  offspring1 = Population(0,0,offspring1_chromo)
  offspring2 = Population(0,0,offspring2_chromo)
  return [offspring1,offspring2]

In [ ]:
def pbx_crossover(population, crossover_rate):
  mating_pool = []
  parent_idx = [i for i in range(population_size)]
  new_population = []
  while len(parent_idx) != 0:
    p1 = random.choice(parent_idx)
    parent_idx.remove(p1)
    p2 = random.choice(parent_idx)
    parent_idx.remove(p2)
    mating_pool.append((p1,p2))

  for p1,p2 in mating_pool:
    if random.uniform(0,1) < crossover_rate:
      new_population = new_population + [*perform_crossover(p1,p2,population)]
    else:
      new_population = new_population + [copy.deepcopy(population[p1]),copy.deepcopy(population[p2])]
  return new_population

In [ ]:
def swap(a,b, chromosome):
  temp = chromosome[a]
  chromosome[a] = chromosome[b]
  chromosome[b] = temp

In [ ]:
def insert_mutation(population, mutation_rate):
  N_DEVICES = len(population[0].chromosome)
  for individual in population:
    if random.uniform(0,1) < mutation_rate:
      random_pos = random.sample(range(N_DEVICES),2)
      min_pos = min(random_pos)
      max_pos = max(random_pos)
      while max_pos != min_pos+1:
        swap(max_pos, max_pos-1, individual.chromosome)
        max_pos = max_pos-1
    else:
      continue

In [ ]:
#mu-lambda survivor strategy
def elitism(parent, offspring):
  parents_offsprings = {}
  new_population = []
  for i in range(population_size):
    parents_offsprings[parent[i]] = parent[i].fitness
    parents_offsprings[offspring[i]] = offspring[i].fitness
  
  for i in range(population_size):
    best_individual = max(parents_offsprings, key = lambda x: parents_offsprings[x])
    del parents_offsprings[best_individual]
    new_population.append(copy.deepcopy(best_individual))
  return new_population

In [ ]:
def node_hopcount(G):
  queue = [GATEWAY_ID]
  N_DEVICES = len(G.nodes)
  hops = 0
  visited = [False for node in range(N_DEVICES)]
  visited[GATEWAY_ID] = True
  hopcounts = {}
  while len(queue) != 0:
    for i in range(len(queue)):
      front = queue.pop(0)
      hopcounts[front] = hops
      for adj_nodes in G.adj[front]:
        if not visited[adj_nodes]:
          visited[adj_nodes] = True
          queue.append(adj_nodes)
    hops = hops + 1
  return hopcounts

In [ ]:
def heuristic_init1(G):
  N_DEVICES = len(G.nodes)
  hopcounts = node_hopcount(G)
  population = []
  for i in range(math.ceil(population_size)):
    temp_list = [(node, -hopcounts[node] + random.uniform(0, tau_max)) for node in range(N_DEVICES)]
    temp_list.sort(key=lambda x: x[1])
    integer_priority = {}
    for priority in range(len(temp_list)):
      integer_priority[temp_list[priority][0]] = priority + random.random()
    population.append(Population(0,0,integer_priority))
  return population

In [ ]:
def Genetic_Algorithm(G, edge_nodes, network_data):
  start = time.time()
  end = time.time()
  converge_iter = 0
  t = 1
  N_DEVICES = len(G.nodes)

  #Heuristic initialization of population
  population = heuristic_init1(G)

  #Evaluate initial population
  # evaluate_population :=> population ,generation_counter, edge_nodes, G, network_data
  evaluate_population(population, t, edge_nodes, G, network_data)

  #initialize global best
  best_individual = population[0] 
  
  while t <= max_generations:

    #survival of the fittest
    new_population = roulette_wheel_sel(population)

    #crossover
    offspring_population = pbx_crossover(new_population, crossover_rate)

    #mutation
    insert_mutation(offspring_population, mutation_rate)

    #evaluate offspring population
    evaluate_population(offspring_population, t, edge_nodes, G, network_data)

    for p in population:
      if p.fitness < best_individual.fitness:
        best_individual = copy.deepcopy(p)
        converge_iter = t
        end = time.time()

    #survival of the fittest
    population = elitism(population, offspring_population)

    t = t+1

  ans = {}
  for f in edge_nodes:
    valid, nodes, indices = decode_path(G, f, best_individual.chromosome)
    indices.append(7)
    a = [i for i in nodes[:-1]]
    b = [j for j in nodes[1:]]
    path = [(i,j,k) for i,j,k in zip(a,b,indices)]
    ans[f] = path
  return [best_individual.objVal, ans, converge_iter, (end-start)]  

In [ ]:
expected_ans = read_data("/content/gdrive/MyDrive/Networks/expected_ans.pkl", False)

In [ ]:
route_optimality_ratio = {}
near_route_optimality_ratio = {}
route_failure_ratio = {}
near_route_failure_ratio = {}
results = {}
percent_gap = {}
iterations_to_converge = {}
time_to_converge = {}
invalid_tests = {}
for N_DEVICES in num_of_nodes:
  correct_solution = 0
  nearly_correct = 0
  total_tests = 0
  invalid_graph = 0
  results[N_DEVICES] = []
  percent_gap[N_DEVICES] = []
  iterations_to_converge[N_DEVICES] = []
  time_to_converge[N_DEVICES] = []
  for t in range(1,test_case_num+1):

    #reading data
    network_data = {}
    G = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/graph.gml",True)
    network_data['tx_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/tx_time.txt", False)
    network_data['max_edge_data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_data_rate.txt",False)
    network_data['cst'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/cst.txt",False)
    network_data['max_flow_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_flow_delays.txt",False)
    network_data['edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_delays.txt",False)
    network_data['max_edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_delays.txt",False)
    network_data['data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/data_rate.txt",False)
    network_data['eh_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/eh_time.txt",False)
    network_data['init_res_energy'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/init_res_energy.txt",False)
    edge_nodes = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_nodes.txt",False)


    #Get 2D coordinates of all the devices
    coordinates = {i:G.nodes()[i]['pos'] for i in G.nodes}

    #creating a list of bidirectional edges
    edge_list = []
    for u,v in G.edges:
      edge_list.append((u,v))
      edge_list.append((v,u))
    if expected_ans[N_DEVICES][t] == -1:
      invalid_graph = invalid_graph + 1
      continue
    optimal_cost, optimal_path, converge_iter, converge_time = Genetic_Algorithm(G, edge_nodes, network_data)
    optimal_cost = round(optimal_cost, 2)
    print(f"num_nodes = {N_DEVICES}, test_case = {t}, actual_cost = {optimal_cost}, optimal_cost = {expected_ans[N_DEVICES][t]}")

    time_to_converge[N_DEVICES].append(converge_time)
    iterations_to_converge[N_DEVICES].append(converge_iter)
    results[N_DEVICES].append([optimal_cost,expected_ans[N_DEVICES][t]])
    percent_gap[N_DEVICES].append(((optimal_cost-expected_ans[N_DEVICES][t])/expected_ans[N_DEVICES][t])*100)
    total_tests = total_tests + 1
    if ((optimal_cost-expected_ans[N_DEVICES][t])/expected_ans[N_DEVICES][t])*100 <= 4:
      nearly_correct = nearly_correct + 1
    if optimal_cost == expected_ans[N_DEVICES][t]:
      correct_solution = correct_solution + 1
  
  #calculating route optimality ratio
  if total_tests == 0:
    continue
  invalid_tests[N_DEVICES] = invalid_graph
  route_optimality_ratio[N_DEVICES] = correct_solution/total_tests
  route_failure_ratio[N_DEVICES] = (total_tests-correct_solution)/total_tests
  near_route_optimality_ratio[N_DEVICES] = nearly_correct/total_tests
  near_route_failure_ratio[N_DEVICES] = nearly_correct/total_tests
  

In [ ]:
#saving results
def save_file(file_name, results):
  with open("/content/gdrive/MyDrive/Networks/" + file_name, 'wb') as file:
    pickle.dump(results, file)
    file.close

In [ ]:
save_file("GA_PX_route_optimality_ratio.pkl", route_optimality_ratio)
save_file("GA_PX_route_failure_ratio.pkl", route_failure_ratio)
save_file("GA_PX_results.pkl", results)
save_file("GA_PX_percent_gap.pkl", percent_gap)
save_file("GA_PX_iterations_to_converge.pkl", iterations_to_converge)
save_file("GA_PX_time_to_converge.pkl", time_to_converge)